# Web scraping av polisens händelser

En vanlig organisation som ingår i journalisternas nyhetsnät är polisen. På polisens hemsida finns en lista över händelser i Sverige (https://polisen.se/aktuellt/polisens-nyheter/). Vi kan förvisso leta bland händelserna manuellt efter något som intresserar oss, men varför inte göra det automatiskt med hjälp av Python?

Vi kan lösa det på två sätt:

1. **Skrapa RSS-flödet**. Vi kan skrapa polisens RSS-flöde med händelser (se dme här: https://polisen.se/aktuellt/rss/). Detta är lättast eftersom [RSS](https://sv.wikipedia.org/wiki/RSS) är en standard för att utbyta information på webbplatser. Då kan vi också använda koden till alla webbplatser som har RSS-flöde. Men polisens RSS-flöde är begränsat, och alla händelser som finns på hemsidan finns inte med i RSS-flödet.
2. **Skrapa webbsidan**. Vi kan skrapa webbsidans HTML-kod. Då kan vi få ned allt som finns på hemsidan. Men om polisen ändrar sin hemsida så kan vårt program slutera fungera.

Vilken ska vi välja? Vi testar båda!


## Först, installera bibliotek

In [1]:
# Först måste vi dock installera de bibliotek vi
# ska använda. Ändra till pip3 om pip inte fungerar.
!pip install BeautifulSoup4
!pip install lxml

## 1. Skrapa RSS-flödet

Se alla RSS-flöden som polisen har: https://polisen.se/aktuellt/rss/

Vi använder detta RSS-flöde (nyheter i hela landet): https://polisen.se/aktuellt/rss/hela-landet/nyheter-hela-landet/

In [2]:
import urllib.request
from lxml import html
from bs4 import BeautifulSoup 

# Funktion för stt skrapa HTML från webbsida.
def scrapewebpage(url):
	# Öppna URL och hämta HTML.
	web = urllib.request.urlopen(url)

	# Kontrollera att inga fel uppstod när URL öppnades.
	if (web.getcode() == 200):
		html = web.read()
		return(html)
	else:
		print("Error %s reading %s" % str(web.getcode()), url)

# Hjälpfunktion som skrapar webbsidor och gör en soppa av allt ihop.
def makesoup(url):
	html = scrapewebpage(url)
	return(BeautifulSoup(html, "lxml"))

In [3]:
# Läs in RSS-flödet och spara det i variablen rss_events.'
rss_events = makesoup("https://polisen.se/aktuellt/rss/hela-landet/handelser-i-hela-landet/")

# Du kan också testa med ett annat RSS-flöde:
#rss_events = makesoup("https://www.svd.se/feed/articles.rss")

# Ta ut alla <item> från RSS-flödet.
rss_items = rss_events.find_all("item")

In [4]:
# Hur många händelser hittade vi?
print("Hittade " + str(len(rss_items)) + " händelser.")

Hittade 200 händelser.


In [5]:
# Visa senaste 10 händelserna. Plocka bort [0:10] för att visa alla.
for item in rss_items[0:10]:
    title = item.find("title").get_text()
    print(title)

19 oktober 14.09, Skadegörelse, Gävle
19 oktober 11.38, Farligt föremål, misstänkt, Lidingö
19 oktober 13.40, Misshandel, grov, Växjö
19 oktober 13.46, Trafikolycka, vilt, Ragunda
19 oktober 13.18, Trafikbrott, Forshaga
19 oktober 12.58, Trafikolycka, Oskarshamn
19 oktober 10.33, Stöld, Ovanåker
19 oktober 12.12, Mord/dråp, försök, Avesta
19 oktober 10.09, Stöld, Östhammar
19 oktober 11.41, Fylleri/LOB, Sundsvall


In [6]:
# En variabel som visar antalet sökresultat.
search_results = 0

# Visa bara trafikolyckor.
for item in rss_items:
    title = item.find("title").get_text()
    description = item.find("description").get_text()

    # Gör om texten till gemener, sök därefter efter trafikolyckor i Göteborg.
    if title.lower().find("trafikolycka") > -1 and title.lower().find("göteborg") > -1:
        search_results = search_results + 1
        print(title + ": " + description)
        print()

print("Hittade {0} resultat.".format(search_results))

19 oktober 11.43, Trafikolycka, personskada, Göteborg: På Önnereds Hamnväg är en bilist och en mopedist inblandade i en olycka.

19 oktober 06.33, Trafikolycka, personskada, Göteborg: På Bredfjällsgatan har två bilar kolliderat.

18 oktober 18.41, Trafikolycka, Göteborg: Kollision på E6 vid Örgrytemotet

18 oktober 16.40, Trafikolycka, Göteborg: Kollission mellan spårvagn och mopedist i Majorna

Hittade 4 resultat.


In [7]:
# Gå igenom varje händelse och plocka ut datum, händelse och stad.
for item in rss_items:
    title = item.find("title").get_text()             # Titel på händelsen
    link = item.find("link").get_text()               # Länk till händelsen på polisens hemsida
    description = item.find("description").get_text() # Beskrivning av händelsen
    
    # Titlarna är skrivna i formatet "2017-11-07 20:40, Trafikolycka, Göteborg".
    # Då kan vi dela upp titeln i tre delar genom komma-tecknet
    # och på så vis plocka ut datum, händelse och ort.
    title_parts = title.split(",")                         # Dela sträng vid varje komma
    title_date = title_parts[0].strip()                    # Datum
    title_event = title_parts[1].strip()                   # Händelse
    title_city = title_parts[len(title_parts) - 1].strip() # Stad
    
    # Plocka ut händelser från Göteborg och Borås.
    if title_city in ["Göteborg", "Borås"]:
        print(title_date + " " + title_city + ", " + title_event + ": " + description)

19 oktober 11.43 Göteborg, Trafikolycka: På Önnereds Hamnväg är en bilist och en mopedist inblandade i en olycka.
18 oktober 21.28 Göteborg, Mord/dråp: På Köpingsgatan har någon skjutit mot en bostad.
19 oktober 06.33 Göteborg, Trafikolycka: På Bredfjällsgatan har två bilar kolliderat.
18 oktober 18.41 Göteborg, Trafikolycka: Kollision på E6 vid Örgrytemotet
18 oktober 16.40 Göteborg, Trafikolycka: Kollission mellan spårvagn och mopedist i Majorna


## 2. Skrapa webbsidan

Vi kan skrapa HTML-koden från webbsidan https://polisen.se/aktuellt/polisens-nyheter/.

Här har vi dock ett problem eftersom polisens hemsida är ganska komplext uppbyggd med både 

- statisk HTML-kod (det man ser på sidan när man högerklickar och väljer "Visa källa").
- dynamisk HTML-kod (det som dyker upp på sidan genom att ett JavaScript-program hämtar information från ett [API](https://en.wikipedia.org/wiki/Application_programming_interface)). Det är detta som ändrar sidans [DOM](https://en.wikipedia.org/wiki/Document_Object_Model).

Men även om det är krångligare för oss att förstå hur sidan är uppbyggd, blir det faktiskt enklare att hämta informationen! Tack vare att polisen använder ett API som hämtar och skickar [JSON](https://en.wikipedia.org/wiki/JSON)-data blir det också enklare för oss att hämta informationen!

In [8]:
# Importera bibliotek vi behöver.
import requests
import time
from lxml import html
from bs4 import BeautifulSoup
import ssl

# Skapa funktion för att kommunicera med polisens API.
def get_police_json(pageindex=1, location=0):
    """ Anropar polisens händelser-API och returnerar JSON. 
    
    location:
        0 = Hela landet
        181 = Göteborg
    """
    url = "https://polisen.se/api/listingservice/items"
    web = requests.post(url, data = {"ContentId": "7069",
                                    "PageIndex": pageindex,
                                    "IncludeChildIndexes": False,
                                    "FilterOnSelectedLocation": (location > 0),
                                    "SelectedLocationId": location,
                                    "GeoAreaType": 1,
                                    "Filter": None, 
                                    "Latitude": 0,
                                    "Longitude":0,
                                    "ListItemType":0,
                                    "PropertyIds": []
                                   })
    json_content = web.json()
    return(json_content)

# Skapa funktion som gör det enkelt att hämta många händelser från polisen.
def get_events(maxpages=10, location=0):
    """ Hämtar händelser från polisen och returnerar lista. """
    l = []
    for pageindex in range(1, maxpages + 1):
        print("Hämtar händelser från sida {0}...".format(pageindex))
        json = get_police_json(pageindex, location)
        for event in json["List"]:
            l.append(event)
        if not json["HasMore"]:
            print("Slut på händelser.")
            break
        time.sleep(1.0)
    return(l)

In [9]:
# Hämta polisens händelser från deras hemsida.
events = get_events(maxpages=10)

Hämtar händelser från sida 1...
Hämtar händelser från sida 2...
Hämtar händelser från sida 3...
Hämtar händelser från sida 4...
Hämtar händelser från sida 5...
Hämtar händelser från sida 6...
Hämtar händelser från sida 7...
Hämtar händelser från sida 8...
Hämtar händelser från sida 9...
Hämtar händelser från sida 10...


In [10]:
# Hur många händelser hämtade vi?
len(events)

250

In [11]:
# Visa den senaste händelsen i listan.
# Notera att Python börjar räkna från 0.
events[0]

{'HasImage': False,
 'HasTeaserText': False,
 'Headline': '19 oktober 14.09, Skadegörelse, Gävle',
 'ImageDescription': '',
 'ImageUrl': '',
 'IsUpdated': False,
 'ListItemType': 4,
 'Location': '',
 'MainContent': '<p>En inringare uppr&ouml;rs &ouml;ver p&aring;g&aring;ende klotter p&aring;&nbsp;ett konstverk&nbsp;i Kvarnparken i G&auml;vle. <br />Under samtalet med RLC-operat&ouml;ren erinrar sig inringaren pl&ouml;tsligt att konstverket &auml;r ett s k klotterplank som det &auml;r till&aring;tet att klottra p&aring;. Inringaren &auml;r &auml;nd&aring; stark motst&aring;ndare till det p&aring;g&aring;ende klottrandet som av allt att d&ouml;ma verkar utf&ouml;ras av en skolklass i regnkl&auml;der.</p>',
 'Organisation': None,
 'Preamble': 'Det klottras på ett klotterplank.',
 'PublishedDate': '19 oktober 14.26',
 'RelatedPages': [],
 'SharedLinksData': {'SharePageByEmailUrl': 'mailto:?subject=Tips%20om%20information%20p%C3%A5%20polisens%20webbplats&body=Hej%2C%0A%0Ajag%20vill%20tipsa%

In [12]:
# Eftersom datan är sparad i en lista, kan vi använda
# en for-loop för att hämta varje händelse i listan.
for e in events:
    print(e["Headline"])

19 oktober 14.09, Skadegörelse, Gävle
19 oktober 11.38, Farligt föremål, misstänkt, Lidingö
19 oktober 13.40, Misshandel, grov, Växjö
19 oktober 13.46, Trafikolycka, vilt, Ragunda
19 oktober 13.18, Trafikbrott, Forshaga
19 oktober 12.58, Trafikolycka, Oskarshamn
19 oktober 10.33, Stöld, Ovanåker
19 oktober 12.12, Mord/dråp, försök, Avesta
19 oktober 10.09, Stöld, Östhammar
19 oktober 11.41, Fylleri/LOB, Sundsvall
19 oktober 12.39, Trafikolycka, Nybro
19 oktober 11.23, Stöld, Västerås
19 oktober 12.12, Rån, Emmaboda
Fem frågor till polisen: ”Min mamma är otroligt rädd för att bli id-kapad”
19 oktober 10.52, Stöld, Örnsköldsvik
19 oktober 08.16, Mord/dråp, Tanum
19 oktober 12.16, Trafikkontroll, Boden
19 oktober 11.43, Trafikolycka, personskada, Göteborg
19 oktober 11.58, Trafikkontroll, Lycksele
19 oktober 11.06, Skadegörelse, Hedemora
19 oktober 11.37, Arbetsplatsolycka, Örebro
19 oktober 11.20, Stöld, Örebro
19 oktober 11.39, Trafikolycka, Tranås
19 oktober 10.06, Stöld/inbrott, Arbog

In [13]:
# Vi kan skapa en enkel sökfunktion som
# letar igenom alla händelser vi samlat in. 
def search(word, events):
    for e in events:
        if e["Headline"].lower().find(word.lower()) > 0:
            print(e["Headline"])

In [14]:
# Sök igenom händelserna efter texten "Göteborg".
search("göteborg", events)

19 oktober 11.43, Trafikolycka, personskada, Göteborg
18 oktober 21.28, Mord/dråp, försök, Göteborg
19 oktober 06.33, Trafikolycka, personskada, Göteborg
18 oktober 18.41, Trafikolycka, Göteborg
18 oktober 16.40, Trafikolycka, Göteborg


In [15]:
# Sök igenom händelserna efter texten "mord".
search("mord", events)

19 oktober 12.12, Mord/dråp, försök, Avesta
19 oktober 08.16, Mord/dråp, Tanum
18 oktober 21.28, Mord/dråp, försök, Göteborg
19 oktober 05.32, Mord/dråp, försök, Stockholms län
18 oktober 13.33, Mord/dråp, försök, Västerås
18 oktober 10.53, Mord/dråp, Trelleborg
18 oktober 03.12, Mord/dråp, Stockholm
17 oktober 23.58, Mord/dråp, försök, Östersund


# Nyhetsrobot – trafikolycka

Utifrån polisens händelser kan vi nu skapa en nyhetsrobot.

Roboten ska skriva om trafikolyckor i Göteborg senaste tiden.

In [16]:
# Gör en sökfunktion som returnerar en lista.
def get_search_result(word, events):
    l = []
    for e in events:
        if e["Headline"].lower().find(word.lower()) > 0:
            l.append(e)
    return(l)
            
# Ta ut allt som har med Göteborg att göra och spara i gbg_list.
gbg_list = get_search_result("göteborg", events)

# Ta sedan ut allt som har med trafikolyckor att göra (från gbg_list) och spara i trafik_list.
trafik_list = get_search_result("trafikolycka", gbg_list)

## Alla trafikolyckor

Skriv en text som visar en lista med alla trafikolyckor som skrapats.

In [17]:
# Skapa variabel med antalet trafikolyckor.
antal_trafikolyckor = len(trafik_list)

# Logiken för vad nyhetsroboten ska skriva.
# Kontrollera först att listan innehåller trafikolyckor.
if antal_trafikolyckor > 0:
    print("Det har skett " + str(antal_trafikolyckor) + " trafikolyckor i Göteborg.")
    print()
    print("Olyckorna skedde vid följande datum:")
    
    # Gå igenom varje trafikolycka i listan, rad för rad.
    for trafik in trafik_list:
        # Plocka ut datumet från rubriken.
        # Rubriken är skriven i formatet "2018-10-19, Händelse, plats"
        # vilket innebär att vi kan dela strängen där komma-tecknet finns
        # och sedan plocka ut första delen som innehåller datumet ("2018-10-19").
        datum = trafik["Headline"]
        datum = datum.split(",")
        datum = datum[0]
        
        # Plocka ut beskrivningen.
        beskrivning = trafik["Preamble"]
        
        # Skriv en punktlista för varje trafikolycka med datum + beskrivning.
        print("- " + datum + " " + beskrivning)
else:
    # Om listan är tom visas detta meddelandet. 
    print("Det har inte skett några trafikolyckor den senaste tiden.")

Det har skett 4 trafikolyckor i Göteborg.

Olyckorna skedde vid följande datum:
- 19 oktober 11.43 På Önnereds Hamnväg är en bilist och en mopedist inblandade i en olycka.
- 19 oktober 06.33 På Bredfjällsgatan har två bilar kolliderat.
- 18 oktober 18.41 Kollision på E6 vid Örgrytemotet
- 18 oktober 16.40 Kollission mellan spårvagn och mopedist i Majorna


## Visa senaste trafikolyckan

I stället för att visa en lista med alla trafikolyckor kan man visa enbart den senaste trafikolyckan.

Det görs genom att ta ut första trafikolyckan från listan med hjälp av `trafik_list[0]`. Kom ihåg att Python börjar räkna från `0`, vilket då representerar det första värdet i listan.

In [19]:
# Skapa variabel med antalet trafikolyckor.
antal_trafikolyckor = len(trafik_list)

# Logiken för vad nyhetsroboten ska skriva.
# Kontrollera först att listan innehåller trafikolyckor.
if antal_trafikolyckor > 0:
    print("Det har skett " + str(antal_trafikolyckor) + " trafikolyckor i Göteborg.")
    
    # Ta ut första värdet från listan.
    trafik = trafik_list[0]
    
    # Plocka ut datumet från rubriken.
    # Rubriken är skriven i formatet "2018-10-19, Händelse, plats"
    # vilket innebär att vi kan dela strängen där komma-tecknet finns
    # och sedan plocka ut första delen som innehåller datumet ("2018-10-19").
    datum = trafik["Headline"]
    datum = datum.split(",")
    datum = datum[0]

    # Plocka ut beskrivningen.
    beskrivning = trafik["Preamble"]
    
    # Komponera ihop en text.
    print("Den senaste olyckan skedde " + datum + ". " + beskrivning)
else:
    # Om listan är tom visas detta meddelandet. 
    print("Det har inte skett några trafikolyckor den senaste tiden.")

Det har skett 4 trafikolyckor i Göteborg.
Den senaste olyckan skedde 19 oktober 11.43. På Önnereds Hamnväg är en bilist och en mopedist inblandade i en olycka.
